# Transformación con petl

![Actividades ETL](./img/etl_actividades.svg)

## Librería petl

- [petl](https://petl.readthedocs.io/en/stable/index.html) es un paquete de propósito general para la extracción, transformación y carga de tablas de datos.
- Diseñado especialmente para trabajar __datos desconocidos, heterogéneos y/o de calidad mixta__.
- __No tiene dependencias__ que no sean módulos propios del nucleo de Python.
- Las _pipelines_ (transformaciones de flujo de datos en un proceso comprendido por varias fases secuenciales) hacen __uso mínimo de la memoria del sistema__ y pueden __escalar a millones de filas si la velocidad no es una prioridad__.
- En general, __los flujos de trabajo no se ejecutan hasta que se soliciten los datos__.
- Soporta programación funcional y orientada a objetos.
-  [Otras alternativas para diseñar procesos ETL](https://petl.readthedocs.io/en/stable/related_work.html)

[Instalación de la librería petl](https://petl.readthedocs.io/en/stable/intro.html#installation)

## Conversión de valores

La función `.convert()` permite __convertir valores invocando funciones__ y los campos afectos como un diccionario. 

Por ejemplo, valores de una columnas desde `str` a `float`:

In [1]:
import petl as etl

niveles = etl.fromxlsx('./dataset/niveles.xlsx')
niveles.look()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |     64.59 |
+--------------+-----------+
| '2012-05-23' |     64.79 |
+--------------+-----------+
| '2012-07-30' |     64.35 |
+--------------+-----------+
| '2012-09-10' |     64.77 |
+--------------+-----------+
| '2012-11-15' |     68.05 |
+--------------+-----------+
...

In [3]:
type(niveles[1][1])

float

In [5]:
n_niveles = etl.convert(niveles, {'Nivel (m)': int})
n_niveles.look()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |        64 |
+--------------+-----------+
| '2012-05-23' |        64 |
+--------------+-----------+
| '2012-07-30' |        64 |
+--------------+-----------+
| '2012-09-10' |        64 |
+--------------+-----------+
| '2012-11-15' |        68 |
+--------------+-----------+
...

In [6]:
type(n_niveles[1][1])

int

También se permite la __sustitución de valores__ invocando los valores como un diccionario cuyo conjunto `key:value` corresponde al origen y destino, respectivamente.

In [7]:
empleados = etl.fromjson('./dataset/empleados.json')
empleados_1 = etl.convert(empleados, 'ciudad', {'Santiago': 'Metropolitana', 'Concepción': 'Biobio'})
empleados_1.look()

+----------+----------------+------+-----------------+--------+-------------+
| nombre   | segundo_nombre | edad | ciudad          | casado | telefono    |
+==========+================+======+=================+========+=============+
| 'Juan'   | None           |   30 | 'Metropolitana' | True   | '987654321' |
+----------+----------------+------+-----------------+--------+-------------+
| 'Sandra' | 'Ariel'        |   24 | 'Biobio'        | False  | '123456789' |
+----------+----------------+------+-----------------+--------+-------------+

Otras [alternativas de conversión de valores](https://petl.readthedocs.io/en/stable/transform.html#converting-values):
- Conversión pasando el método y sus parámetros.
- Conversión condicional.
- Conversión accediendo a datos de la misma fila.
- Conversión de todo a número.
- Conversión por interpolación.
- ...

## Filtros y búsqueda

El segundo argumento de la función `.select()` puede ser una expresión con forma de condición, expresado como un literal de tipo `str`.

In [8]:
niveles_2012 = etl.select(niveles, "{Fecha}>'2012' and {Fecha}<'2013'")
niveles_2012.lookall()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |     64.59 |
+--------------+-----------+
| '2012-05-23' |     64.79 |
+--------------+-----------+
| '2012-07-30' |     64.35 |
+--------------+-----------+
| '2012-09-10' |     64.77 |
+--------------+-----------+
| '2012-11-15' |     68.05 |
+--------------+-----------+

In [9]:
niveles.lookall()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |     64.59 |
+--------------+-----------+
| '2012-05-23' |     64.79 |
+--------------+-----------+
| '2012-07-30' |     64.35 |
+--------------+-----------+
| '2012-09-10' |     64.77 |
+--------------+-----------+
| '2012-11-15' |     68.05 |
+--------------+-----------+
| '2013-03-14' |     73.33 |
+--------------+-----------+
| '2013-05-16' |     75.19 |
+--------------+-----------+
| '2013-07-29' | None      |
+--------------+-----------+
| '2013-09-11' |     72.68 |
+--------------+-----------+
| '2014-03-27' |     85.12 |
+--------------+-----------+
| '2014-05-22' |     83.77 |
+--------------+-----------+
| '2014-07-25' |     86.19 |
+--------------+-----------+
| '2014-09-30' |     88.57 |
+--------------+-----------+
| '2014-11-25' | None      |
+--------------+-----------+
| '2015-05-22' |    102.14 |
+--------------+-----------+

In [10]:
filtro_73 = etl.select(niveles, "{Nivel (m)} != 73.33") # ! =
filtro_73.lookall()

+--------------+-----------+
| Fecha        | Nivel (m) |
+==============+===========+
| '2012-01-23' |     64.59 |
+--------------+-----------+
| '2012-05-23' |     64.79 |
+--------------+-----------+
| '2012-07-30' |     64.35 |
+--------------+-----------+
| '2012-09-10' |     64.77 |
+--------------+-----------+
| '2012-11-15' |     68.05 |
+--------------+-----------+
| '2013-05-16' |     75.19 |
+--------------+-----------+
| '2013-07-29' | None      |
+--------------+-----------+
| '2013-09-11' |     72.68 |
+--------------+-----------+
| '2014-03-27' |     85.12 |
+--------------+-----------+
| '2014-05-22' |     83.77 |
+--------------+-----------+
| '2014-07-25' |     86.19 |
+--------------+-----------+
| '2014-09-30' |     88.57 |
+--------------+-----------+
| '2014-11-25' | None      |
+--------------+-----------+
| '2015-05-22' |    102.14 |
+--------------+-----------+

Es posible __mapear campos de valores de una tabla en un diccionario__. La función `.facet()` recibe como segundo argumento la columna que será mapeada como claves del diccionario.

In [11]:
empleados.look()

+----------+----------------+------+--------------+--------+-------------+
| nombre   | segundo_nombre | edad | ciudad       | casado | telefono    |
+==========+================+======+==============+========+=============+
| 'Juan'   | None           |   30 | 'Santiago'   | True   | '987654321' |
+----------+----------------+------+--------------+--------+-------------+
| 'Sandra' | 'Ariel'        |   24 | 'Concepción' | False  | '123456789' |
+----------+----------------+------+--------------+--------+-------------+

In [13]:
nombre = etl.facet(empleados, 'nombre')
nombre.keys()

dict_keys(['Sandra', 'Juan'])

In [17]:
nombre['Juan'].look()

+--------+----------------+------+------------+--------+-------------+
| nombre | segundo_nombre | edad | ciudad     | casado | telefono    |
+========+================+======+============+========+=============+
| 'Juan' | None           |   30 | 'Santiago' | True   | '987654321' |
+--------+----------------+------+------------+--------+-------------+

## Uso de expresiones regulares

A partir de [expresiones regulares](https://docs.python.org/3/library/re.html#regular-expression-syntax) se realiza una busqueda de valores y __retorna las filas que coinciden con un patrón__.

In [18]:
import pandas as pd

url = 'http://www.sismologia.cl/ultimos_sismos.html'
tablas = pd.read_html(url)
sismos_df = tablas[0]
sismos = etl.fromdataframe(sismos_df)

etl.cut(sismos, 'Referencia Geográfica').look()

+--------------------------------------+
| Referencia Geográfica                |
+======================================+
| '73 km al SE de Socaire'             |
+--------------------------------------+
| '60 km al O de San Pedro de Atacama' |
+--------------------------------------+
| '43 km al NE de Calama'              |
+--------------------------------------+
| '76 km al N de Huasco'               |
+--------------------------------------+
| '42 km al O de La Ligua'             |
+--------------------------------------+
...

En __cualquier lugar de un registro__ (fila):

In [22]:
sismos.lookall()

+-----------------------+-----------------------+---------+----------+------------------+----------+--------------------------------------+
| Fecha Local           | Fecha UTC             | Latitud | Longitud | Profundidad [Km] | Magnitud | Referencia Geográfica                |
+=======================+=======================+=========+==========+==================+==========+======================================+
| '2021-10-21 07:57:27' | '2021-10-21 10:57:27' | -24.127 |  -67.484 |            221.3 | '3.9 Ml' | '73 km al SE de Socaire'             |
+-----------------------+-----------------------+---------+----------+------------------+----------+--------------------------------------+
| '2021-10-21 07:01:50' | '2021-10-21 10:01:50' | -23.133 |  -68.739 |            102.1 | '3.5 Ml' | '60 km al O de San Pedro de Atacama' |
+-----------------------+-----------------------+---------+----------+------------------+----------+--------------------------------------+
| '2021-10-21 04:31:

In [21]:
sismos_68 = etl.search(sismos, '68')
sismos_68

Fecha Local,Fecha UTC,Latitud,Longitud,Profundidad [Km],Magnitud,Referencia Geográfica
2021-10-21 07:01:50,2021-10-21 10:01:50,-23.133,-68.739,102.1,3.5 Ml,60 km al O de San Pedro de Atacama
2021-10-21 04:31:17,2021-10-21 07:31:17,-22.227,-68.607,121.6,3.1 Ml,43 km al NE de Calama
2021-10-20 14:59:50,2021-10-20 17:59:50,-22.112,-68.721,116.5,3.1 Ml,45 km al NE de Calama


Dentro de __un campo específico__:

In [32]:
sismos_tongoy = etl.search(sismos, 'Referencia Geográfica', 'Tongoy')
sismos_tongoy.cut('Magnitud', 'Referencia Geográfica').look()

+----------+-------------------------+
| Magnitud | Referencia Geográfica   |
+==========+=========================+
| '3.3 Ml' | '75 km al NO de Tongoy' |
+----------+-------------------------+
| '2.6 Ml' | '72 km al NO de Tongoy' |
+----------+-------------------------+

### Ordenamiento

Por defecto, la función `.sort()` realiza sobre una tabla un ordenamiento léxico:

In [25]:
catalogo = etl.fromxml('./dataset/cd_catalog.xml', 'CD', 
                       {'cia': 'COMPANY', 'artista': 'ARTIST', 'pais': 'COUNTRY'})
catalogo.look()

+-------------------+------------------+-------+
| artista           | cia              | pais  |
+===================+==================+=======+
| 'Bob Dylan'       | 'Columbia'       | 'USA' |
+-------------------+------------------+-------+
| 'Bonnie Tyler'    | 'CBS Records'    | 'UK'  |
+-------------------+------------------+-------+
| 'Dolly Parton'    | 'RCA'            | 'USA' |
+-------------------+------------------+-------+
| 'Gary Moore'      | 'Virgin records' | 'UK'  |
+-------------------+------------------+-------+
| 'Eros Ramazzotti' | 'BMG'            | 'EU'  |
+-------------------+------------------+-------+
...

In [27]:
catalogo_ordenado = etl.sort(catalogo, reverse=True)
etl.lookall(catalogo_ordenado)

+---------------------+------------------+----------+
| artista             | cia              | pais     |
+=====================+==================+==========+
| 'Will Smith'        | 'Columbia'       | 'USA'    |
+---------------------+------------------+----------+
| 'Van Morrison'      | 'Polydor'        | 'UK'     |
+---------------------+------------------+----------+
| 'Tina Turner'       | 'Capitol'        | 'UK'     |
+---------------------+------------------+----------+
| 'The Communards'    | 'London'         | 'UK'     |
+---------------------+------------------+----------+
| "T'Pau"             | 'Siren'          | 'UK'     |
+---------------------+------------------+----------+
| 'Simply Red'        | 'Elektra'        | 'EU'     |
+---------------------+------------------+----------+
| 'Savage Rose'       | 'Mega'           | 'EU'     |
+---------------------+------------------+----------+
| 'Sam Brown'         | 'A and M'        | 'UK'     |
+---------------------+-----

Ingresando como argumento el nombre de __uno o más campos__:
- El parámetro `reverse=True` permite ejecutar un ordenamiento en sentido contrario.

In [29]:
sismos_ordenados = etl.sort(sismos, key=['Magnitud'], reverse=False)
sismos_ordenados.cut('Magnitud', 'Referencia Geográfica').look()

+----------+--------------------------------+
| Magnitud | Referencia Geográfica          |
+==========+================================+
| '2.6 Ml' | '55 km al NO de Los Vilos'     |
+----------+--------------------------------+
| '2.6 Ml' | '31 km al S de Pica'           |
+----------+--------------------------------+
| '2.6 Ml' | '72 km al NO de Tongoy'        |
+----------+--------------------------------+
| '2.7 Ml' | '159 km al NO de Constitución' |
+----------+--------------------------------+
| '2.7 Ml' | '18 km al SE de Pica'          |
+----------+--------------------------------+
...

También es posible __comprobar el ordenamiento de una tabla__:

In [31]:
etl.issorted(sismos_ordenados, key='Magnitud')

True

Otras operaciones de transformación:
- `mergesort()`
- `join()`
- `unjoin()` (split)
- `duplicates()`
- `unique()`
- `aggregate()`
- `reshape()`
- `transpose()`
- `pivot()`
- `fill...()`


## Extras

Encontrar __mínimo y máximo__:

In [33]:
mn, mx = etl.limits(sismos, 'Magnitud')
print('Máx: {}\nMín: {}'.format(mx, mn))

Máx: 3.9 Ml
Mín: 2.6 Ml


Algunos __estadísticos básicos__:

In [34]:
estadisticos_niveles = etl.stats(niveles, 'Nivel (m)')
estadisticos_niveles

stats(count=13, errors=2, sum=993.5399999999998, min=64.35, max=102.14, mean=76.42615384615385, pvariance=129.69419289940822, pstdev=11.388335826599434)

In [37]:
for estadistico in estadisticos_niveles:
    print(estadistico)

13
2
993.5399999999998
64.35
102.14
76.42615384615385
129.69419289940822
11.388335826599434


__Contar valores__:

In [55]:
etl.valuecounts(catalogo, 'cia').lookall()

+------------------+-------+----------------------+
| cia              | count | frequency            |
+==================+=======+======================+
| 'Polydor'        |     3 |  0.11538461538461539 |
+------------------+-------+----------------------+
| 'Columbia'       |     2 |  0.07692307692307693 |
+------------------+-------+----------------------+
| 'Atlantic'       |     2 |  0.07692307692307693 |
+------------------+-------+----------------------+
| 'CBS Records'    |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'RCA'            |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'Virgin records' |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'BMG'            |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'CBS'            |     1 | 0.038461538461538464 |
+------------------+-------+----------------------+
| 'Pickwick'